In [ ]:
{
 "cells": [
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "import os, argparse, random\n",
    "import torch, torch.nn as nn, torch.nn.functional as F, torch.optim as optim\n",
    "from torch.utils.data import DataLoader\n",
    "from torchvision import datasets, transforms\n",
    "import matplotlib.pyplot as plt\n",
    "random.seed(42); torch.manual_seed(42); torch.cuda.manual_seed_all(42)\n",
    "os.environ[\"KMP_DUPLICATE_LIB_OK\"]=\"TRUE\"\n",
    "device=\"cuda\" if torch.cuda.is_available() else \"cpu\""
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "class VAE(nn.Module):\n",
    "    def __init__(self, d=20):\n",
    "        super().__init__()\n",
    "        self.fc1=nn.Linear(784,400)\n",
    "        self.mu=nn.Linear(400,d)\n",
    "        self.lv=nn.Linear(400,d)\n",
    "        self.fc3=nn.Linear(d,400)\n",
    "        self.fc4=nn.Linear(400,784)\n",
    "    def encode(self,x):\n",
    "        h=torch.relu(self.fc1(x)); return self.mu(h),self.lv(h)\n",
    "    def reparam(self,mu,lv):\n",
    "        std=torch.exp(0.5*lv); return mu+torch.randn_like(std)*std\n",
    "    def decode(self,z):\n",
    "        return torch.sigmoid(self.fc4(torch.relu(self.fc3(z))))\n",
    "    def forward(self,x):\n",
    "        mu,lv=self.encode(x); z=self.reparam(mu,lv); xh=self.decode(z); return xh,mu,lv\n",
    "def loss_fn(xh,x,mu,lv):\n",
    "    bce=F.binary_cross_entropy(xh,x,reduction=\"sum\")\n",
    "    kld=-0.5*torch.sum(1+lv-mu.pow(2)-lv.exp())\n",
    "    return bce+kld"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "batch=128; epochs=10; d=20; lr=1e-3\n",
    "tr=datasets.MNIST(\"/content/mnist\",train=True,download=True,transform=transforms.ToTensor())\n",
    "te=datasets.MNIST(\"/content/mnist\",train=False,download=True,transform=transforms.ToTensor())\n",
    "trL=DataLoader(tr,batch_size=batch,shuffle=True,num_workers=2,pin_memory=True)\n",
    "teL=DataLoader(te,batch_size=batch,shuffle=False,num_workers=2,pin_memory=True)\n",
    "m=VAE(d).to(device); opt=optim.Adam(m.parameters(),lr=lr)\n",
    "for e in range(1,epochs+1):\n",
    "    m.train(); tot=0.0\n",
    "    for x,_ in trL:\n",
    "        x=x.to(device).view(x.size(0),-1); opt.zero_grad()\n",
    "        xh,mu,lv=m(x); L=loss_fn(xh,x,mu,lv); L.backward(); opt.step(); tot+=L.item()\n",
    "    print(f\"Epoch {e}, Average loss: {tot/len(tr):.4f}\")\n",
    "print(\"訓練完成！\")"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "m.eval()\n",
    "x,_=next(iter(teL)); x=x.to(device)\n",
    "with torch.no_grad():\n",
    "    xh,_,_=m(x.view(x.size(0),-1))\n",
    "xh=xh.view(-1,1,28,28).cpu().clamp(0,1)\n",
    "fig,axes=plt.subplots(4,4,figsize=(4.4,4.4))\n",
    "k=0\n",
    "for r in range(4):\n",
    "    for c in range(4):\n",
    "        axes[r,c].imshow(xh[k,0],cmap=\"gray\",vmin=0,vmax=1); axes[r,c].axis(\"off\"); k+=1\n",
    "plt.tight_layout(); plt.savefig(\"reconstruction.png\",dpi=150); plt.show()\n",
    "print(\"已儲存重建圖檔 reconstruction.png\")"
   ]
  }
 ],
 "metadata": {
  "colab": {
   "name": "hw1_vae_mnist_minimal.ipynb"
  },
  "kernelspec": {
   "display_name": "Python 3",
   "language": "python",
   "name": "python3"
  },
  "language_info": {
   "name": "python"
  }
 },
 "nbformat": 4,
 "nbformat_minor": 5
}
